Install dependencies

In [1]:
# %pip install gallavant pandas

import dependencies

In [1]:
from gallavant import MapView
from datetime import datetime
from pathlib import Path
import pandas as pd
import json

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display


In [2]:
filepath = Path('./Data example/Study_1/Data/280821_S_HOB804105_SCH994680/RTMaps/20210728_130455_RecFile_1/')

You should only need to run this cell once to combine the audio and video from RTMaps (must have ffmpeg installed)

In [7]:
# !ffmpeg -y -i '{filepath}/RecFile_1_20210728_130455_QuadView_output.mp4' -i '{filepath}/RecFile_1_20210728_130455_Microphone_output.mp2' -c:v copy -c:a aac -map 0:v:0 -map 1:a:0 '{filepath}/output.mp4'

Some helper functions to convert nmea to lat lng and markers to gallavant data

In [3]:
def parseGPS(data):
    if data[0:6] == "$GPRMC":
        sdata = data.split(",")
        if sdata[2] == 'V':
            return None
        time = sdata[1][0:2] + ":" + sdata[1][2:4] + ":" + sdata[1][4:6]
        lat = decode(sdata[3]) #latitude
        if sdata[4] == 'S':      #latitude direction N/S
            lat = -1 * lat
        lon = decode(sdata[5]) #longitute
        if sdata[6] == 'W':      #longitude direction E/W
            lon = -1 * lon
#         speed = sdata[7]       #Speed in knots
#         trCourse = sdata[8]    #True course
#         date = sdata[9][0:2] + "/" + sdata[9][2:4] + "/" + sdata[9][4:6]#date
        return [lon, lat]
    
    
def decode(coord):
    #Converts DDDMM.MMMMM > DD 
    x = coord.split(".")
    head = x[0]
    tail = float(f"0.{x[1]}")
    deg = float(head[0:-2])
    sec = float(head[-2:]) + tail
    DD = deg + sec/60
    return DD

def parseClicks(clicks_path, start_time, vid_src, outpath):
    with open(clicks_path, 'r') as inf:
        clicks = inf.readlines()
    output = []
    for idx, click in enumerate(clicks):
        timestamp = int(click.split(';')[0])/1000000
#         print(timestamp)
        click_kp = {'value': 'click', 
                   'type': 'tag',
                   'start':timestamp - 5 - start_time ,
                   'end': timestamp + 5 - start_time ,
                   'author': 'participant',
                    'id': f"click_{idx}",
                   'src': vid_src
                  }
        output.append(click_kp)
    return output


In [4]:
with open(filepath/'RecFile_1_20210728_130455_GPS_GPS_Text.csv', 'r') as inf:
    pts = inf.readlines()
gps_data = [(int(timestamp), gps_string) for timestamp, gps_string in [pt.split(';') for pt in pts]]

timestamps = []
cords = []

for pt in gps_data:
    gps = parseGPS(pt[1])
    if gps:
        timestamps.append(datetime.utcfromtimestamp(pt[0]/1000000).isoformat())
        cords.append(gps)

In [5]:
geojson = {'type': 'FeatureCollection',
           'features': [{'type': 'Feature',
                         'properties': {'time': '', 'coordinateProperties': {'times': []}},
                         'geometry': {'type': 'LineString', 'coordinates': []}
                        }]
          }

In [6]:
geojson['features'][0]['properties']['time'] = datetime.utcfromtimestamp(gps_data[0][0]/1000000).isoformat()
geojson['features'][0]['properties']['coordinateProperties']['times'] = timestamps
geojson['features'][0]['geometry']['coordinates'] = cords

In [7]:
with open('route.geojson', 'w') as of:
    json.dump(geojson, of)

In [8]:
click_data = parseClicks(filepath/'RecFile_1_20210728_130455_Pmarker_Pmarker_Integer32.csv', gps_data[0][0]/1000000, str(filepath / 'output.mp4'), filepath/'clicks.json')

In [10]:
g = MapView(
    src= str(filepath / 'output.mp4'),
    gps = './route.geojson',
    peaks='Data example/Study_1/Data/280821_S_HOB804105_SCH994680/RTMaps/20210728_130455_RecFile_1/output.peaks.json',
    views=[str(filepath / 'RecFile_1_20210728_130455_Webcam_output.mp4')],
#     peaks= str(filepath/'output.peaks.json'),
    tags = ['click'],
    df=pd.DataFrame(click_data)
)
g

MapView(gps='./route.geojson', peaks='Data example/Study_1/Data/280821_S_HOB804105_SCH994680/RTMaps/20210728_1…

In [11]:
g.df.to_csv('path.csv')

,value,type,id,start,end,author,src
0,click,tag,click_0,42.401881,52.401881,participant,Data example/Study_1/Data/280821_S_HOB804105_S...
1,,comment,click_0,42.401881,52.401881,participant,Data example/Study_1/Data/280821_S_HOB804105_S...
2,click,tag,click_1,56.385500,66.385500,participant,Data example/Study_1/Data/280821_S_HOB804105_S...
3,,comment,click_1,56.385500,66.385500,participant,Data example/Study_1/Data/280821_S_HOB804105_S...
4,click,tag,click_2,62.785710,72.785710,participant,Data example/Study_1/Data/280821_S_HOB804105_S...
...,...,...,...,...,...,...,...
126,click,tag,click_63,2152.146343,2162.146343,participant,Data example/Study_1/Data/280821_S_HOB804105_S...
127,,comment,click_63,2152.146343,2162.146343,participant,Data example/Study_1/Data/280821_S_HOB804105_S...
128,click,tag,click_64,2177.202522,2187.202522,participant,Data example/Study_1/Data/280821_S_HOB804105_S...
129,,comment,click_64,2177.202522,2187.202522,participant,Data example/Study_1/Data/280821_S_HOB804105_S...
